In [1]:
# import pandas library

import pandas as pd

In [2]:
# load dataset

dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
# Categorical input data

dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [23]:
# Drop UserID column 
# axis =1 for column
# axis =0 for row

dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
# Balanced/Imbalanced data
# Output - Purchased 
# 1. How many class we have to classify
# 2. Data - Balanced/Imbalanced
# Balanced data - learn without partiallity

# value_counts - Python - value_counts() function returns object containing counts of unique values.
# the class 0 learns with 257 data
# the class 1 learns with 143 data
# Hence, the Given class is imbalanced
# class 0 gives the best result than class 1


dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [24]:
# view dataset column names

dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [25]:
# Split Input data

independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [7]:
# split Output Data

dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
#The shape() method returns the shape of an array i.e. the number of elements in each dimension. 

independent.shape

# 400 rows × 4 columns

(400, 4)

In [27]:
# split training and testing data

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent, dependent, test_size=1/3, random_state=0)

In [28]:
# Standardization

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [29]:
# GridSearchCV Model Creation - RF
# classifier=RandomForestClassifier(n_estimatores=10,criterion='entropy,random_state=0)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid={ 'n_estimators': [200, 500],
            'criterion' :['gini', 'entropy']}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


F:\Anaconda\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [200, 500]},
             scoring='f1_weighted', verbose=3)

In [30]:
# Hyper Tuning parameter

result=grid.cv_results_

# Evaluation metrics

y_pred=grid.predict(x_test)

In [31]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[79,  6],
       [ 4, 45]], dtype=int64)

In [32]:
# Classification report
# classify the class based on the test data 
# y_test=actual data
# y_pred=predicted data

from sklearn.metrics import classification_report
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [33]:
# View in dataFrame

table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.741139,0.020969,0.065159,0.008698,gini,200,"{'criterion': 'gini', 'n_estimators': 200}",0.847141,0.888107,0.851527,0.925272,0.943699,0.891149,0.038579,4
1,1.866842,0.026554,0.127121,0.004488,gini,500,"{'criterion': 'gini', 'n_estimators': 500}",0.867478,0.907177,0.851527,0.925272,0.943699,0.899031,0.034648,2
2,0.766324,0.016329,0.060762,0.009615,entropy,200,"{'criterion': 'entropy', 'n_estimators': 200}",0.867478,0.870047,0.851527,0.925272,0.962264,0.895318,0.041738,3
3,1.724531,0.323191,0.106934,0.019316,entropy,500,"{'criterion': 'entropy', 'n_estimators': 500}",0.867478,0.907177,0.851527,0.925272,0.962264,0.902744,0.039841,1


In [34]:
# f1 score

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'n_estimators': 500} 0.9256751954513147


In [35]:
# roc_auc_score

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9593037214885954

In [38]:
# Output from Enduser

age=int(input("Age :"))
estimated_salary=float(input("Estimated_Salary :"))
gender_male=int(input("gender_male 1 or 0 :"))

# Output Prediction

Prediction=grid.predict([[age,estimated_salary,gender_male]])
print("Future Prediction {}".format(Prediction))

Age :12
Estimated_Salary :0
gender_male 1 or 0 :0
Future Prediction [1]
